# SCRAPPING DATA

In [1]:
pip install google-api-python-client pandas

In [ ]:
import pandas as pd
from googleapiclient.discovery import build

# --- KONFIGURASI ---
API_KEY = "API_KEY"
VIDEO_ID = "sPyb2e0l5Vw"


def get_comments(video_id, api_key):
    # Inisialisasi layanan YouTube API
    youtube = build('youtube', 'v3', developerKey=api_key)
    comments_list = []
    next_page_token = None

    print(f"Memulai scraping komentar untuk Video ID: {video_id}")

    while True:
        try:
            # Panggil API untuk mengambil thread komentar
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=100,  # Maksimum yang diizinkan per permintaan
                pageToken=next_page_token
            )

            response = request.execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']

                # Ekstrak data yang Anda inginkan
                comments_list.append({
                    'author': comment['authorDisplayName'],
                    'comment_text': comment['textDisplay'],
                    'published_at': comment['publishedAt'],
                    'like_count': comment['likeCount']
                })

            # Periksa apakah ada halaman berikutnya
            next_page_token = response.get('nextPageToken')

            if not next_page_token:
                break

            print(f"  Sudah mendapatkan {len(comments_list)} komentar. Melanjutkan ke halaman berikutnya...")

        except Exception as e:
            print(f"Terjadi kesalahan: {e}")
            break

    print(f"Scraping selesai. Total komentar: {len(comments_list)}")
    return comments_list

# Jalankan fungsi
comments_data = get_comments(VIDEO_ID, API_KEY)

# Konversi ke DataFrame Pandas
df = pd.DataFrame(comments_data)

# Simpan ke file CSV
output_filename = f"SEDIH4_comments.csv"
df.to_csv(output_filename, index=False, encoding='utf-8')

print(f"Data telah berhasil disimpan ke file: {output_filename}")

Memulai scraping komentar untuk Video ID: sPyb2e0l5Vw
  Sudah mendapatkan 100 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 200 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 300 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 400 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 500 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 600 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 700 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 800 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 900 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 1000 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 1100 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 1200 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan 1300 komentar. Melanjutkan ke halaman berikutnya...
  Sudah mendapatkan

MARAH: 34889+128
SEDIH: 354+414+879+...+1788
BAHAGIA: 27780
TERKEJUT: 1583+134
TAKUT: 140+3016+2207
CINTA: 407+8+100+24+920+1678

# PRE-PROCESSING

In [22]:
import pandas as pd
import os


files_to_merge = [
    "BAHAGIA_comments.csv", "CINTA1_comments.csv", "CINTA_comments.csv",
    "CINTA2_comments.csv", "CINTA3_comments.csv", "CINTA4_comments.csv",
    "CINTA5_comments.csv", "MARAH_comments.csv", "MARAH1_comments.csv", "SEDIH_comments.csv",
    "SEDIH1_comments.csv", "SEDIH2_comments", "SEDIH3_comments.csv", "SEDIH4_comments.csv",
    "TAKUT_comments.csv", "TAKUT1_comments.csv", "TAKUT2_comments.csv", "TERKEJUT_comments.csv",
    "TERKEJUT1_comments"
]

# Siapkan list kosong untuk menampung semua DataFrame
all_dataframes = []
missing_files = []
read_success_count = 0

print("Memulai proses penggabungan file komentar emosi...")

# Looping untuk membaca dan memproses setiap file
for file in files_to_merge:
    # Beberapa file di daftar Anda tidak memiliki ekstensi .csv (misalnya 'SEDIH2_comments')
    # Kita tambahkan penanganan untuk kasus ini.
    if not file.lower().endswith('.csv'):
        # Coba tambahkan .csv jika belum ada
        file_name_with_ext = file + ".csv"
    else:
        file_name_with_ext = file

    try:
        # Baca file CSV
        df = pd.read_csv(file_name_with_ext)
        all_dataframes.append(df)
        read_success_count += 1
        # print(f"  [BERHASIL] Memuat: {file_name_with_ext}")

    except FileNotFoundError:
        # Jika file tidak ditemukan, catat dan lewati
        missing_files.append(file_name_with_ext)
    except pd.errors.EmptyDataError:
        print(f"  [PERINGATAN] File {file_name_with_ext} kosong. Melewati.")
    except Exception as e:
        print(f"  [ERROR] Gagal memuat {file_name_with_ext}: {e}")

# Gabungkan semua DataFrame dalam list menjadi satu DataFrame besar
if all_dataframes:
    final_combined_df_emosi = pd.concat(all_dataframes, ignore_index=True)

    # Simpan hasil gabungan ke file baru
    output_filename_emosi = 'DATA_KOMENTAR_EMOSI_GABUNGAN.csv'
    final_combined_df_emosi.to_csv(output_filename_emosi, index=False, encoding='utf-8')

    print("\n--- Ringkasan Komentar Emosi ---")
    print(f"✅ BERHASIL menggabungkan {read_success_count} file menjadi: {output_filename_emosi}")
    print(f"Total baris data: {len(final_combined_df_emosi)}")
    if missing_files:
        print("\n⚠️ PERHATIAN: File berikut tidak ditemukan dan dilewati:")
        for missing in missing_files:
            print(f"- {missing}")
else:
    print("\n❌ GAGAL: Tidak ada file komentar emosi yang berhasil dimuat.")

Memulai proses penggabungan file komentar emosi...

--- Ringkasan Komentar Emosi ---
✅ BERHASIL menggabungkan 18 file menjadi: DATA_KOMENTAR_EMOSI_GABUNGAN.csv
Total baris data: 76751

⚠️ PERHATIAN: File berikut tidak ditemukan dan dilewati:
- CINTA2_comments.csv


In [24]:
import pandas as pd
import re

# 1. Tentukan nama file input dan output
file_input = 'DATA_KOMENTAR_EMOSI_GABUNGAN.csv'
file_output = 'DATA_KOMENTAR_EMOSI_CLEANED.csv'

# 2. Baca file CSV
try:
    df = pd.read_csv(file_input)
    print(f"Berhasil memuat file: {file_input}")
except FileNotFoundError:
    print(f"ERROR: File '{file_input}' tidak ditemukan. Pastikan file berada di direktori yang sama.")
    exit()

# Kolom yang akan diproses
kolom_teks = 'comment_text'

# --- 3. Case Folding (huruf kecil semua) ---
print("\n[LANGKAH 1/3] Menerapkan Case Folding...")
# Pastikan kolom adalah string sebelum di-lowercase
df[kolom_teks] = df[kolom_teks].astype(str).str.lower()

# --- 4. Fungsi untuk preprocessing bertahap ---
def clean_text(text):
    """Fungsi untuk membersihkan teks dari URL, tag HTML, simbol, dan spasi berlebih."""
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)    # hapus URL
    text = re.sub(r'<.*?>', '', text)                      # hapus HTML tag
    # Hapus angka & simbol, hanya menyisakan huruf (a-z) dan spasi
    # PERHATIAN: Ini juga akan menghapus karakter non-ASCII seperti emoji atau karakter bahasa lain.
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()               # hapus spasi berlebih
    return text

# --- 5. Terapkan Fungsi Cleaning ke Kolom Teks ---
print("[LANGKAH 2/3] Menerapkan fungsi cleaning kustom...")
# Buat kolom baru untuk menyimpan hasil cleaning, agar kolom asli tetap ada (opsional, tapi disarankan)
df['TEXT_CLEANED'] = df[kolom_teks].apply(clean_text)

# --- 6. Simpan Data Hasil Cleaning ---
print("[LANGKAH 3/3] Menyimpan data yang sudah dibersihkan...")
df.to_csv(file_output, index=False, encoding='utf-8')

print("\n--- RINGKASAN PROSES ---")
print(f"Proses Case Folding dan Cleaning Teks Selesai!")
print(f"Data disimpan ke file baru: {file_output}")
print("\nContoh Perubahan Data:")
print("Kolom asli ('TEXT'):")
print(df[kolom_teks].head())
print("\nKolom hasil cleaning ('TEXT_CLEANED'):")
print(df['TEXT_CLEANED'].head())

Berhasil memuat file: DATA_KOMENTAR_EMOSI_GABUNGAN.csv

[LANGKAH 1/3] Menerapkan Case Folding...
[LANGKAH 2/3] Menerapkan fungsi cleaning kustom...
[LANGKAH 3/3] Menyimpan data yang sudah dibersihkan...

--- RINGKASAN PROSES ---
Proses Case Folding dan Cleaning Teks Selesai!
Data disimpan ke file baru: DATA_KOMENTAR_EMOSI_CLEANED.csv

Contoh Perubahan Data:
Kolom asli ('TEXT'):
0                                       sy fan om dedy
1         semoga jirayut sama halda jodoh yang sakinah
2                                       ini lucu tau😂😂
3      disaat halda dah sama sadam, aku masih disini 😢
4    itu cewek berisik banget, sok asik tapi garing...
Name: comment_text, dtype: object

Kolom hasil cleaning ('TEXT_CLEANED'):
0                                       sy fan om dedy
1         semoga jirayut sama halda jodoh yang sakinah
2                                         ini lucu tau
3         disaat halda dah sama sadam aku masih disini
4    itu cewek berisik banget sok asik tapi garing